In [2]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`


In [3]:
using BenchmarkTools

In [32]:
n = 40
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 20
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 100, tol = 1e-10, verbose=0);
slater = Slater(rhf);
nn = NeuralNetwork([Dense(640), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);

In [9]:
2^20

1048576

In [33]:
optimizer = ADAM(wf, lr = 0.01, max_iter = 1, tol = 1e-6);
metro = Importance(equils = 0, samples = 2^20, step = 0.01);
@time wf_final, wf_opt, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = false);

In [31]:
metro = Importance(equils = 0, samples = 2^20, step = 0.01);
@btime result = blocking(wf, ham, metro, nthreads = 12);

  2.540 s (2416 allocations: 92.63 MiB)
